# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        imp_l = 1 - (l_c**2).sum(axis = 1) / (l_s**2)[:,0]
        imp_r = 1 - (r_c**2).sum(axis = 1) / (r_s**2)[:,0]
        return l_s[:,0] / (l_s[:,0] + r_s[:,0]) * imp_l + r_s[:,0] / (l_s[:,0] + r_s[:,0]) * imp_r
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        eps = 1e-5
        elems = l_s[0,0]+r_s[0,0]
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        l_c = np.clip(l_c, eps, elems - eps)
        r_c = np.clip(r_c, eps, elems - eps)
        imp_l = (- l_c  * 
                 (np.log(l_c / (l_s)))).sum(axis = 1)
        imp_r = (- r_c  * 
                 (np.log(r_c / (r_s)))).sum(axis = 1) 
        return (imp_l + imp_r) / (l_s[:,0] + r_s[:,0]) 
    
    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_s[:,0] / (l_s[:,0] + r_s[:,0]) * (1 - l_c.max(axis = 1) / l_c.sum(axis = 1)) + r_s[:,0] / (l_s[:,0] + r_s[:,0]) * (1 - r_c.max(axis = 1)/r_c.sum(axis = 1))



    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[round(np.log(n_feature))] # Ваш код
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[round(np.log(n_feature))] # Ваш код

    def __get_feature_ids_N(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:n_feature] # Ваш код
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # x - значения фичи для набора, y - соответствующие классы. Сортирует массив по x.  
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # Количество различных классов
        class_number = np.unique(y).shape[0] 
        # Проблема в том, что такой class_number не даст работать при >2 классов.
        # Почему? Получим IndexError в one_hot_code. Fix:
        class_number = y.max() + 1
        #print 'class number', class_number, np.unique(y), np.unique(sorted_y)
        # Что делает этот блок кода?
        # Первые и последние min_samples_split точек убираются(они неразделимы), для остальных
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        # Индексы элеменетов, на которых будет смена классов
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        #print 'splitted_sorted_y', splitted_sorted_y
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # Блоки одинакового класса(первые неделимые элементы выделены в отдельный блок,последний блок игнорируем)
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        # Создает бинарный array. Количество строк==количество возможных разбиений. 
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        # Кодируем единицей столбец класса, которому принадлежит элемент, после которого будет смена класса.
        #print np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        
        # Находим размер блоков одного класса
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # Для первого блока(в котором могут лежать разные классы) восстанавливаем кол-во элементов классов
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # *_class_count - array(строки - разбиения, столбцы - количество элементов данного класса)
        # *_sizes - размеры левого и правого блоков
        l_class_count = np.cumsum(class_increments, axis=0) 
        #print np.bincount(sorted_y), l_class_count
        r_class_count = np.bincount(sorted_y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Считаем impurity_measures для каждого разбиения
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        # Берем номер разбиения с минимальными impurity
        idx = np.argmin(gs)
        # Что делает этот блок кода?
        # Берет индекс элемента, являющегося первым во втором блоке
        left_el_id = l_sizes[idx][0]
        #print 'left_id', left_el_id, 
        
        #print (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
        # Возвращаем impurity_measure, threshold = среднее между граничными значениями блоков
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        if(depth == self.max_depth)  or (x.shape[0] < 2 * self.min_samples_split + 2) or ((np.bincount(y).max()/y.shape) > self.sufficient_share):
            self.tree[node_id] = self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y)/y.shape
            return
        features = self.get_feature_ids(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:,i],y) for i in features])
        best_split_feature = thresholds[:,0].argmin()
        impurity, threshold = thresholds[best_split_feature]
        left_data, right_data, left_classes, right_classes = self.__div_samples(x, y, 
                                                                                features[best_split_feature], 
                                                                                threshold)
        if (left_data.shape[0] < self.min_samples_split) or (right_data.shape[0] < self.min_samples_split):
            self.tree[node_id] = self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y)/y.shape
            return
        self.tree[node_id] = self.NON_LEAF_TYPE, features[best_split_feature], threshold
        self.__fit_node(left_data, left_classes, 2 * node_id + 1, depth + 1, pred_f)
        self.__fit_node(right_data, right_classes, 2 * node_id + 2, depth + 1, pred_f)              
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 
        #Node: [0] - LEAF_TYPE/NON_LEAF_TYPE
        #[1] - Top class/feature_id
        #[2] - Variances of classes/threshold
        
        

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [5]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [6]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.391345024109
0.969997882843


## Проверка качества работы

In [7]:
gkf = KFold(n_splits=5, shuffle=True)

In [8]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.93219422965
0.934480751642
0.930406585183
0.929367257005
0.934103853989


In [9]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.890163798121
0.892242454477
0.892450320113
0.891951442588
0.893277345861
